In [67]:
import glob
import json
import csv
import os
import pandas as pd

In [68]:
def loadResultPaths(path, dataset, instance):
    all_instance_path = glob.glob(path + dataset + "/" + instance[:-4] + "/*.csv")
    all_instance_path = [i.replace("\\","/") for i in all_instance_path]
    # print(all_instance_path)
    csv_files = [eachpath.split("/")[-1] for eachpath in all_instance_path]
    # print(csv_files)
    return all_instance_path, csv_files

In [69]:
def cleanResult(csv_file_path):
    loaded_result = pd.read_csv(csv_file_path)

    min_column = loaded_result['min']

    def clean_row(inp):
        out = inp.replace("[","").replace("]","").strip().split(" ")
        return out

    min_dist = [float(clean_row(i)[-1]) for i in min_column]
    return min_dist[-1]


In [70]:
print(cleanResult("../results/result_GA/19-4-2022/A/A-n32-k5/A-n32-k5_pop200_crossProb0.8_mutProb0.1_numGen5000_seed0.csv"))

901.10063206


In [71]:
with open("./analyze_config.json", 'r') as f:
        config = json.load(f)

In [72]:
def write_to_csv(filename, data):
    file = open(filename, 'w+', newline ='')
  
    # writing the data into the file
    with file:    
        write = csv.writer(file)
        write.writerows(map(lambda x: [x], data))

In [73]:
def analyzeInstance(config, dataset, instance, data):
    filename = config["output_path"] + dataset + "/" + instance[:-4] + ".csv"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    write_to_csv(filename, data)


In [74]:
def analyzeAllInstance(config):
    for dataset in config["input_result"]["set"]:
        for instance in config["instance"][dataset]:
            instance_path, csv_files = loadResultPaths(config["input_result"]["input_path"], 
                                                        dataset, instance)
            # Analyze all
            res = []
            for eachpath in instance_path:
                res.append(cleanResult(eachpath))
            # print(res)
            res.append(min(res))
            res.append(sum(res)/len(res))
            analyzeInstance(config, dataset, instance, res)

In [75]:
analyzeAllInstance(config)